In [1]:
from google.colab import files
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [2]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['Tweets.csv']))

In [3]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from string import punctuation
punct = list(punctuation)

from nltk.stem import PorterStemmer
ps = PorterStemmer()

import re

from nltk.corpus import wordnet

from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [51]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [41]:
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [46]:
def pre_process(tweet):
  tweet=tweet.lower()
  tweet = re.sub(r'\d+', '', tweet)
  tokens = word_tokenize(tweet)
  tokens=[i for i in tokens if i.isalpha()]
  cleaned_tokens = [token for token in tokens if token not in stop_words ]

  #stems=[ps.stem(token) for token in cleaned_tokens]
  tags=nltk.pos_tag(cleaned_tokens)
  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), tags))

  lemmatized=[]
  for word in wordnet_tagged:
      if word[1]!=None:
          lemmatized.append(lemmatizer.lemmatize(word[0],word[1]))
      else:
          lemmatized.append(lemmatizer.lemmatize(word[0]))
  return lemmatized






In [40]:
pre_process("tweet 65 hfhf kehd #4uh5 % 789 ")

['tweet', 'hfhf', 'kehd', 'uh']

In [53]:
df["cleaned_tweets"]

0                           [virginamerica, dhepburn, say]
1        [virginamerica, plus, added, commercial, exper...
2        [virginamerica, today, must, mean, need, take,...
3        [virginamerica, really, aggressive, blast, obn...
4                 [virginamerica, really, big, bad, thing]
                               ...                        
14635    [americanair, thank, get, different, flight, c...
14636    [americanair, leave, minute, late, flight, war...
14637    [americanair, please, bring, american, airline...
14638    [americanair, money, change, flight, answer, p...
14639    [americanair, ppl, need, know, many, seat, nex...
Name: cleaned_tweets, Length: 14640, dtype: object

In [52]:
df["cleaned_tweets"]=df["text"].map(lambda x: pre_process(x))